In [1]:
import pandas as pd
from orion.data import load_signal, load_anomalies
from lintel.utils import *

In [2]:
df = pd.DataFrame(columns = ['signal', 'sequence', 'ano_indices'])

In [3]:
df

,signal,sequence,ano_indices


In [4]:
train_data = load_signal('E-1')

In [5]:
train_label = load_anomalies('E-1')

In [6]:
train_label

,start,end
0,1393027200,1393675200
1,1406203200,1416484800


In [7]:
print(train_data[(train_data.timestamp >= train_label.start[0]) & (train_data.timestamp <= train_label.end[0])].index.to_list())

[7880, 7881, 7882, 7883, 7884, 7885, 7886, 7887, 7888, 7889, 7890, 7891, 7892, 7893, 7894, 7895, 7896, 7897, 7898, 7899, 7900, 7901, 7902, 7903, 7904, 7905, 7906, 7907, 7908, 7909, 7910]


In [8]:
e7 = create_str_seq(train_data, 7900, 8700)

In [9]:
e7_ano_ind = create_str_ind(8163-7900, 8443-7900)

In [10]:
e1 = create_str_seq(train_data, 7600, 8200)

In [11]:
e1_ano_ind = create_str_ind(7880-7600, 7910-7600)

In [12]:
m = get_message_with_example(e1, e7, e7_ano_ind)

In [13]:
m

[{'role': 'system',
  'content': "You are a helpful assistant that performs time series anomaly detection. The user will provide an example of a sequence and a list of indices that are anomalous. Then the user will provide sequence and you will be asked to give a list of ranges that are anomalous in the sequence. The sequences are represented by decimal strings separated by commas. Please give a list of indices are anomalous in the  following sequence without producing any additional text. Do not say anything like 'the anomalous ranges in the sequence are', just return the numbers."},
 {'role': 'user',
  'content': 'Sequence: 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 2 0 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 

In [14]:
ans = get_model_response(m)

In [15]:
e1_ano_ind

'2 8 0 , 2 8 1 , 2 8 2 , 2 8 3 , 2 8 4 , 2 8 5 , 2 8 6 , 2 8 7 , 2 8 8 , 2 8 9 , 2 9 0 , 2 9 1 , 2 9 2 , 2 9 3 , 2 9 4 , 2 9 5 , 2 9 6 , 2 9 7 , 2 9 8 , 2 9 9 , 3 0 0 , 3 0 1 , 3 0 2 , 3 0 3 , 3 0 4 , 3 0 5 , 3 0 6 , 3 0 7 , 3 0 8 , 3 0 9 , 3 1 0 , '

In [16]:
l = create_list_ind(ans)

In [19]:
l

[11, 109, 116, 214]

In [ ]:
# loop 100 times 